In [2]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from citipy import citipy

# # Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# For formatting date
from datetime import datetime



In [4]:

# Create a set of 2000 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)  #the zip object packs each pair of lats and lngs into a tuple with index 0,1

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)   #unzip them into a list

# for coordinate in coordinates, display the list feeding in index values:
#     print(coordinate[0], coordinate[1])

In [5]:
# Create a list for holding the cities.  
cities = []
# Identify the nearest city for each latitude and longitude combination (use citipy).
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name  #like a look up, given coords x,y, look up city
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
# print(cities)

8

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# print(url)



In [7]:
## Perform an API call with the OpenWeatherMap and retrieve the following info:

# Latitude and longitude# Maximum temperature
# Percent humidity
# Percent cloudiness
# Wind speed
# Weather description (for example, clouds, fog, light rain, clear sky)

In [31]:
# Create an empty list to hold the weather data. 
city_data = []

#Print the beginning of the logging.
print("Beginning Data Retrieval")
print("------------------------")

#Create Counters
record_count = 1
set_count = 1   #because we're batching in sets of 50 


# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):   
        set_count += 1   #increment the set count
        record_count = 1 #at this point you're on the 1st record for the new set
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Display the URL for each city you're processing
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        # Get the data elements that you want.
        city_lat = city_weather["coord"]["lat"]
      
        city_lng = city_weather["coord"]["lon"]
       
        city_max_temp = city_weather["main"]["temp_max"]
       
        city_humidity = city_weather["main"]["humidity"]
         
        city_clouds = city_weather["clouds"]["all"]
        
        city_wind = city_weather["wind"]["speed"]
        
        city_country = city_weather["sys"]["country"]
        
        city_description = city_weather["weather"][0]["description"]

      # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
    #   Append the city information into city_data list.  Here we are creating an array of dictionaries. 
        city_data.append({"City": city.title(),
                              "Cloudiness":city_clouds,
                              "Country": city_country,
                              "Date": city_date,
                              "Humidity": city_humidity,
                              "Lat":city_lat,
                              "Lng": city_lng,
                              "Max Temp": city_max_temp,
                              "Wind Speed":city_wind,
                              "Description":city_description})

  

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
    print("-----------------------------")
    print("Data Retrieval Complete      ")
    print("-----------------------------")




Beginning Data Retrieval
------------------------
Processing Record 1 of Set 1 | brotas
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 2 of Set 1 | arraial do cabo
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 3 of Set 1 | qaanaaq
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 4 of Set 1 | provideniya
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 5 of Set 1 | mataura
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 6 of Set 1 | cape town
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 7 of Set 1 | lompoc
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 8 of Set 1 | kait

In [32]:
# Add the data to a new DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Description
0,Brotas,19,BR,2022-07-27 18:28:39,26,-22.2842,-48.1267,82.81,4.63,few clouds
1,Arraial Do Cabo,0,BR,2022-07-27 18:24:15,69,-22.9661,-42.0278,78.76,18.41,clear sky
2,Qaanaaq,67,GL,2022-07-27 18:28:40,78,77.4840,-69.3632,40.91,7.58,broken clouds
3,Provideniya,100,RU,2022-07-27 18:26:06,77,64.3833,-173.3000,45.82,6.35,overcast clouds
4,Mataura,100,NZ,2022-07-27 18:28:41,99,-46.1927,168.8643,47.48,1.68,light rain
5,Cape Town,0,ZA,2022-07-27 18:28:41,72,-33.9258,18.4232,60.91,11.50,clear sky
6,Lompoc,20,US,2022-07-27 18:24:03,76,34.6391,-120.4579,79.70,8.05,few clouds
7,Kaitangata,100,NZ,2022-07-27 18:28:42,95,-46.2817,169.8464,44.40,5.30,light rain


In [34]:
# Export the new DataFrame as a CSV file named WeathePy_Database.csv into the Weather_Database folder

#  Set location for the 
weather_data_csv = "Weather_Database/WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(weather_data_csv, index_label="City_ID")